In [1]:
import sys
# import os
import time

import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal
from collections import namedtuple, deque
import numpy as np
import random
import matplotlib.pyplot as plt
import model
import imutils
from torchvision import transforms
import time

In [ ]:
# 创建机器人
from jetbot import Robot, Camera, bgr8_to_jpeg
robot = Robot()
# 设置灯光
from RGB_Lib import Programing_RGB
RGB = Programing_RGB()
RGB.OFF_ALL_RGB()  # 关闭所有灯光，如果出现流水灯则表示需要重新摆放位置

# 显示电量
from Battery_Vol_Lib import BatteryLevel
batteryLevel = BatteryLevel()
print(f"电量：{batteryLevel.Update()}")

# 创建相机实例
camera = Camera.instance(width=720, height=720)

11.778885630498534
电量：Battery_Medium


In [ ]:
class image_net():
    def __init__(self):
        model_path = '/home/jetbot/car_train/image_code/workdir/Res18_ep200_lr1.0E-03_bs64/model_para.pth'

        self.device = device
        self.net = model.resnet18(n_classes=4, input_channels=4).to(device)
        print(device)
        self.net.load_state_dict(torch.load(model_path))
        self.transform = transforms.Compose([transforms.ToTensor()])
        
    def get_object_position(self, fig):
        fig = self.preprocess(fig)        
        fig = self.transform(fig).to(device)
        fig = fig.unsqueeze(0)
        begin_time = time.time()
        pred = self.net(fig)
        end_time = time.time()
        # print('net time:{}s'.format((end_time-begin_time)/1000))        
        pred[:,2] = (pred[:,2] - 0.5) * 2
        pred[:,3] = pred[:,3] * 2
        print('pred:',pred[:,2:])
        return pred[:,2:]
    def preprocess(self, img):
        greenLower = (29, 86, 6)
        greenUpper = (64, 255, 255)        
        width, height = img.shape[:2]
        blurred = cv2.GaussianBlur(img, (11, 11), 0)        
        hsv = cv2.cvtColor(blurred, cv2.COLOR_RGB2HSV)
        # cv2.imwrite('./hsv.png', hsv)        
        mask = cv2.inRange(hsv, greenLower, greenUpper)
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)
        # cv2.imwrite('./mask.png', mask)        
        
        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
                                cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        center = None


        ret_mask = np.zeros_like(mask)
        if len(cnts) > 0:
            c = max(cnts, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

            # To see the centroid clearly
            if radius > 10:
                # cv2.circle(img, (int(x), int(y)), int(radius), (0, 255, 255), 5)
                # cv2.imwrite("circled_frame.png", cv2.resize(img, (int(height / 2), int(width / 2))))
                # cv2.circle(img, center, 5, (0, 0, 255), -1)
                cv2.circle(ret_mask, (int(x), int(y)), int(radius), 255, -1)

        # cv2.imwrite('ret_mask.png', ret_mask)
        # cv2.imwrite('./preprocess.png', img)           

        mask_img = np.concatenate((img, ret_mask[:,:,None]), axis=2)
        return mask_img

In [ ]:
class Env():
    def __init__(self):
        self.dim_state = 2
        # 动作维度，左右轮速度
        self.dim_action = 2
        self.action_min = -1.2
        self.action_max = 1.2

    def get_image(self):
        fig = camera.value
        sensorImage = cv2.resize(fig, (720,720))  # 因为cv2版本问题可能会报错
        return sensorImage

    def close(self):
        sim.simxStopSimulation(self.clientId, sim.simx_opmode_blocking)

    def step(self, action, i, img_net):
#         time6 = time.time()
#         object_position = img_net.get_object_position(self.get_image())
#         time7 = time.time()
#         print('image_net_time:', (time7-time6)/1000)
        object_position = [[1,1]]
        robot.set_motors(float(action[0,0]), float(action[0,1]))
        time.sleep(1.0)
        robot.stop()
        print('object_position:', object_position)
        dis_x = object_position[0][0]
        dis_z = object_position[0][1] - 0.6
        reward = -(dis_x ** 2 + dis_z ** 2 + 0.01 * i)
        if abs(dis_x) < 0.1 and abs(dis_z) < 0.1:
            done = True
            reward = 0
        else:
            done = False
        next_state = object_position
        return next_state, reward, done
    
class ReplayBeffer():
    def __init__(self, buffer_maxlen):
        self.buffer = deque(maxlen=buffer_maxlen)

    def push(self, data):
        self.buffer.append(data)

    def sample(self, batch_size):
        state_list = []
        action_list = []
        reward_list = []
        next_state_list = []
        done_list = []

        batch = random.sample(self.buffer, batch_size)
        for experience in batch:
            s, a, r, n_s, d = experience
            # state, action, reward, next_state, done

            state_list.append(s)
            action_list.append(a)
            reward_list.append(r)
            next_state_list.append(n_s)
            done_list.append(d)

            

        return torch.FloatTensor(state_list).to(device), \
               torch.FloatTensor(action_list).to(device), \
               torch.FloatTensor(reward_list).unsqueeze(-1).to(device), \
               torch.FloatTensor(next_state_list).to(device), \
               torch.FloatTensor(done_list).unsqueeze(-1).to(device)

    def buffer_len(self):
        return len(self.buffer)


# Value Net
class ValueNet(nn.Module):
    def __init__(self, state_dim, edge=3e-3):
        super(ValueNet, self).__init__()
        self.linear1 = nn.Linear(state_dim, 256)
        self.linear2 = nn.Linear(256, 256)
        self.linear3 = nn.Linear(256, 2)

        self.linear3.weight.data.uniform_(-edge, edge)
        self.linear3.bias.data.uniform_(-edge, edge)

    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)

        return x


# Soft Q Net
class SoftQNet(nn.Module):
    def __init__(self, state_dim, action_dim, edge=3e-3):
        super(SoftQNet, self).__init__()
        self.linear1 = nn.Linear(state_dim + action_dim, 256)
        self.linear2 = nn.Linear(256, 256)
        self.linear3 = nn.Linear(256, 2)

        self.linear3.weight.data.uniform_(-edge, edge)
        self.linear3.bias.data.uniform_(-edge, edge)

    def forward(self, state, action):
        x = torch.cat([state, action], 1)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)

        return x


# Policy Net
class PolicyNet(nn.Module):
    def __init__(self, state_dim, action_dim, log_std_min=-20, log_std_max=2, edge=3e-3):
        super(PolicyNet, self).__init__()
        self.log_std_min = log_std_min
        self.log_std_max = log_std_max

        self.linear1 = nn.Linear(state_dim, 256)
        self.linear2 = nn.Linear(256, 256)

        self.mean_linear = nn.Linear(256, action_dim)
        self.mean_linear.weight.data.uniform_(-edge, edge)
        self.mean_linear.bias.data.uniform_(-edge, edge)

        self.log_std_linear = nn.Linear(256, action_dim)
        self.log_std_linear.weight.data.uniform_(-edge, edge)
        self.log_std_linear.bias.data.uniform_(-edge, edge)

    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))

        mean = self.mean_linear(x)
        log_std = self.log_std_linear(x)
        log_std = torch.clamp(log_std, self.log_std_min, self.log_std_max)

        return mean, log_std

    def action(self, state):
        state = torch.FloatTensor(state).to(device)
        mean, log_std = self.forward(state)
        std = log_std.exp()
        normal = Normal(mean, std)

        z = normal.sample()
        action = torch.tanh(z).detach().cpu().numpy()
        return action

    # Use re-parameterization tick
    def evaluate(self, state, epsilon=1e-6):
        mean, log_std = self.forward(state)
        std = log_std.exp()
        normal = Normal(mean, std)
        noise = Normal(0, 1)

        z = noise.sample()
        action = torch.tanh(mean + std * z.to(device))
        log_prob = normal.log_prob(mean + std * z.to(device)) - torch.log(1 - action.pow(2) + epsilon)

        return action, log_prob

class SAC:
    def __init__(self, env, gamma, tau, buffer_maxlen, value_lr, q_lr, policy_lr):

        self.env = env
        self.state_dim = env.dim_state
        self.action_dim = env.dim_action

        # hyperparameters
        self.gamma = gamma
        self.tau = tau

        # initialize networks
        self.value_net = ValueNet(self.state_dim).to(device)
        self.target_value_net = ValueNet(self.state_dim).to(device)
        self.q1_net = SoftQNet(self.state_dim, self.action_dim).to(device)
        self.q2_net = SoftQNet(self.state_dim, self.action_dim).to(device)
        self.policy_net = PolicyNet(self.state_dim, self.action_dim).to(device)

        # Load the target value network parameters
        for target_param, param in zip(self.target_value_net.parameters(), self.value_net.parameters()):
            target_param.data.copy_(self.tau * param + (1 - self.tau) * target_param)

        # Initialize the optimizer
        self.value_optimizer = optim.Adam(self.value_net.parameters(), lr=value_lr)
        self.q1_optimizer = optim.Adam(self.q1_net.parameters(), lr=q_lr)
        self.q2_optimizer = optim.Adam(self.q2_net.parameters(), lr=q_lr)
        self.policy_optimizer = optim.Adam(self.policy_net.parameters(), lr=policy_lr)

        # Initialize thebuffer
        self.buffer = ReplayBeffer(buffer_maxlen)

    def get_action(self, state):
        action = self.policy_net.action(state)
        return action

    def update(self, batch_size):
        state, action, reward, next_state, done = self.buffer.sample(batch_size)
        new_action, log_prob = self.policy_net.evaluate(state)

        # V value loss
        value = self.value_net(state)
        new_q1_value = self.q1_net(state, new_action)
        new_q2_value = self.q2_net(state, new_action)
        next_value = torch.min(new_q1_value, new_q2_value) - log_prob
        value_loss = F.mse_loss(value, next_value.detach())

        # Soft q  loss
        q1_value = self.q1_net(state, action)
        q2_value = self.q2_net(state, action)
        target_value = self.target_value_net(next_state)
        target_q_value = reward + done * self.gamma * target_value
        q1_value_loss = F.mse_loss(q1_value, target_q_value.detach())
        q2_value_loss = F.mse_loss(q2_value, target_q_value.detach())

        # Policy loss
        policy_loss = (log_prob - torch.min(new_q1_value, new_q2_value)).mean()

        # Update Policy
        self.policy_optimizer.zero_grad()
        policy_loss.backward()
        self.policy_optimizer.step()

        # Update v
        self.value_optimizer.zero_grad()
        value_loss.backward()
        self.value_optimizer.step()

        # Update Soft q
        self.q1_optimizer.zero_grad()
        self.q2_optimizer.zero_grad()
        q1_value_loss.backward()
        q2_value_loss.backward()
        self.q1_optimizer.step()
        self.q2_optimizer.step()

        # Update target networks
        for target_param, param in zip(self.target_value_net.parameters(), self.value_net.parameters()):
            target_param.data.copy_(self.tau * param + (1 - self.tau) * target_param)

In [ ]:
def main(env, agent, Episode, batch_size):
    Return = []
    action_range = [env.action_min, env.action_max]
    img_net = image_net()

    for episode in range(Episode):
        score = 0
        state = img_net.get_object_position(env.get_image())  # 重新摆放之后获取目标位置
#         state = [[1,1]]
        for i in range(300):
            time1 = time.time()
            action = agent.get_action(state)
            # action output range[-1,1],expand to allowable range
            action_in = action * (action_range[1] - action_range[0]) + (action_range[1] + action_range[0])
            next_state, reward, done= env.step(action_in, i, img_net)
            done_mask = 0.0 if done else 1.0
            agent.buffer.push((state, action, reward, next_state, done_mask))
            state = next_state

            score += reward
            if done:
                break
            if agent.buffer.buffer_len() > 500:
                agent.update(batch_size)

        print("episode:{}, Return:{}, buffer_capacity:{}".format(episode, score, agent.buffer.buffer_len()))
        Return.append(score)
        RGB.Set_WaterfallLight_RGB()  # 设置灯光
        time.sleep(5.0)  # 等待五秒，重新摆放位置，开始下一轮探索
        RGB.OFF_ALL_RGB()
        if  batteryLevel.Update() == "Battery_Low":
            RGB.Set_All_RGB(0xFF, 0x00, 0x00)  # 红灯亮起
            print("低电量！")
    env.close()
    plt.plot(Return)
    plt.ylabel('Return')
    plt.xlabel("Episode")
    plt.grid(True)
    plt.show()

In [ ]:
if __name__ == '__main__':
    env = Env()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device = torch.device('cuda')
    device = "cpu"
    # Params
    tau = 0.01
    gamma = 0.99
    q_lr = 3e-3
    value_lr = 3e-3
    policy_lr = 3e-3
    buffer_maxlen = 50000

    Episode = 100
    batch_size = 128
    quit()
    agent = SAC(env, gamma, tau, buffer_maxlen, value_lr, q_lr, policy_lr)
    main(env, agent, Episode, batch_size)